In [1]:
cd ..

/home/abhishekabhishek/git/AQT-KimGroup


This notebook can be used to run different quantum circuits (e.g. GHZ, W) on IBMQ simulators or hardware, and construct a dataset of POVM measurements which can be used to perform quantum state tomography using attention-based models.

In [2]:
from qiskit import IBMQ, QuantumCircuit
from qiskit.quantum_info import DensityMatrix
from qiskit_experiments.library.tomography import StateTomographyAnalysis
import numpy as np
import os, time

from circuits import state_prep

%matplotlib inline

Setup access to IBM

In [3]:
# Setup account
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')

# Find available devices
devices = provider.backends(operational=True)
for x in devices:
    print(x.name(), x.configuration().n_qubits, x.status().pending_jobs)

ibmq_qasm_simulator 32 2
ibmq_lima 5 153
ibmq_belem 5 8
ibmq_quito 5 24
simulator_statevector 32 2
simulator_mps 100 2
simulator_extended_stabilizer 63 2
simulator_stabilizer 5000 2
ibmq_manila 5 9
ibm_nairobi 7 22
ibm_oslo 7 54


In [4]:
# Use one of the backends above
REAL = False
if REAL:
    sim_backend = provider.backend.ibmq_belem
else:
    sim_backend = provider.backend.ibmq_qasm_simulator
    
print(f'Using backend {sim_backend}')

Using backend ibmq_qasm_simulator


Use the qiskit_experiments library to collect the tomography data

In [5]:
from qiskit_experiments.library import StateTomography

In [30]:
# System setup
n_qubits = 6
n_shots = 100

# Initialize the circuit
circ = QuantumCircuit(n_qubits)

# Choose which type of state to prepare
# One of ghz, w, bisep, random
state = 'bisep'

In [31]:
if state == 'ghz':
    circ.h(0)
    for idx in range(n_qubits-1):
        circ.cx(idx, idx+1)
elif state == 'w':
    if n_qubits == 2:
        circ.ry(1.9106332362490184, 0)
        circ.cu((np.pi/2), (np.pi/2), (np.pi/2), 0, 0, 1)
        circ.cx(1, 2)
        circ.cx(0, 1)
        circ.x(0)
    else:
        circ = state_prep.create_w_state(n_qubits)
elif state == 'bisep':
    if n_qubits == 3:
        circ.h(0)
        circ.h(1)
        for idx in range(1, n_qubits-1):
            circ.cx(idx, idx+1)
    elif n_qubits == 6:
        circ.h(0)
        for idx in range(int(n_qubits/2)-1):
            circ.cx(idx, idx+1)
        circ.h(3)
        for idx in range(int(n_qubits/2), n_qubits-1):
            circ.cx(idx, idx+1)
    else:
        raise NotImplementedError
elif state == 'random':
    for idx in range(n_qubits):
        circ.h(idx)
        
# Draw the circuit
circ.draw() 

┌───┐          
q_0: ┤ H ├──■───────
     └───┘┌─┴─┐     
q_1: ─────┤ X ├──■──
          └───┘┌─┴─┐
q_2: ──────────┤ X ├
     ┌───┐     └───┘
q_3: ┤ H ├──■───────
     └───┘┌─┴─┐     
q_4: ─────┤ X ├──■──
          └───┘┌─┴─┐
q_5: ──────────┤ X ├
               └───┘

Start saving density matrices to the disk

In [24]:
# save options
overwrite = True
data_dir = 'data'
if REAL:
    state_dir = data_dir + f'/{state}_{n_qubits}_{sim_backend.name()}'
else:
    state_dir = data_dir + f'/{state}_{n_qubits}'

if not os.path.isdir(data_dir):
    print(f'{data_dir} does not exist. Creating the directory.')
    os.mkdir(f'{data_dir}')
    
if not os.path.isdir(state_dir):
    print(f'{state_dir} does not exist. Creating the directory.')
    os.mkdir(f'{state_dir}')

In [25]:
ideal_dm_np = DensityMatrix(circ).data
ideal_dm_path = f'{state_dir}/{n_qubits}_{(3**n_qubits)*n_shots}_ideal_dm.npy'
if os.path.exists(ideal_dm_path):
    if overwrite:
        print('overwriting existing ideal dm')
        np.save(ideal_dm_path, ideal_dm_np)
else:
    print('saving new ideal dm')
    np.save(ideal_dm_path, ideal_dm_np)

overwriting existing ideal dm


In [26]:
t1 = time.time()

# No. of tomography circuits to draw
n_draw = 1

# POVM to use
povm = 'pauli6'

# Initalize the tomography circuits
qst_exp = StateTomography(circ)

# Use standard linear inversion fitter
qst_exp.analysis.set_options(fitter='linear_inversion')

qst_data = qst_exp.run(sim_backend, seed_simulation=100, 
                       shots=n_shots).block_for_results()

# Draw the tomogrpahy circuits
for circuit in qst_exp.circuits()[:n_draw]:
    print(circuit.draw())
    
t = divmod(time.time() - t1, 60)
print(f'Linear inversion : took {t[0]} minutes {t[1]} seconds')

     ┌───┐      ░ ┌────────────┐ ░ ┌─┐      
q_0: ┤ H ├──────░─┤ PauliMeasZ ├─░─┤M├──────
     ├───┤      ░ ├────────────┤ ░ └╥┘┌─┐   
q_1: ┤ H ├──■───░─┤ PauliMeasZ ├─░──╫─┤M├───
     └───┘┌─┴─┐ ░ ├────────────┤ ░  ║ └╥┘┌─┐
q_2: ─────┤ X ├─░─┤ PauliMeasZ ├─░──╫──╫─┤M├
          └───┘ ░ └────────────┘ ░  ║  ║ └╥┘
c_0: ═══════════════════════════════╩══╬══╬═
                                       ║  ║ 
c_1: ══════════════════════════════════╩══╬═
                                          ║ 
c_2: ═════════════════════════════════════╩═
                                            
Linear inversion : took 0.0 minutes 6.642184495925903 seconds


In [27]:
# Save the linear inversion dm to the disk
overwrite_linear = True
linear_dm_np = qst_data.analysis_results("state").value.data
linear_dm_path = f'{state_dir}/{n_qubits}_{(3**n_qubits)*n_shots}_linear_dm.npy'

if os.path.exists(linear_dm_path):
    if overwrite_linear:
        print('overwriting existing linear dm')
        np.save(linear_dm_path, linear_dm_np)
else:
    print('saving new linear dm')
    np.save(linear_dm_path, linear_dm_np)

overwriting existing linear dm


Save the POVM measurment data to the disk

In [28]:
# Get the results from the experiments
counts_dict = {}
header_dict = {}

idx = 0
for job_id in qst_data.job_ids:
    result = sim_backend.retrieve_job(job_id).result()
    counts, exp_results = result.get_counts(), result.results
    
    for count, exp_result in zip(counts, exp_results):
        counts_dict[idx] = count
        header_dict[idx] = exp_result.header.name.split('_')[1][1:-1].split(', ')
        idx+= 1

In [29]:
circuits = qst_exp.circuits()
for i in range(3):
    print(circuits[i].draw())
    print(counts_dict[i], header_dict[i])

     ┌───┐      ░ ┌────────────┐ ░ ┌─┐      
q_0: ┤ H ├──────░─┤ PauliMeasZ ├─░─┤M├──────
     ├───┤      ░ ├────────────┤ ░ └╥┘┌─┐   
q_1: ┤ H ├──■───░─┤ PauliMeasZ ├─░──╫─┤M├───
     └───┘┌─┴─┐ ░ ├────────────┤ ░  ║ └╥┘┌─┐
q_2: ─────┤ X ├─░─┤ PauliMeasZ ├─░──╫──╫─┤M├
          └───┘ ░ └────────────┘ ░  ║  ║ └╥┘
c_0: ═══════════════════════════════╩══╬══╬═
                                       ║  ║ 
c_1: ══════════════════════════════════╩══╬═
                                          ║ 
c_2: ═════════════════════════════════════╩═
                                            
{'000': 23, '001': 33, '110': 23, '111': 21} ['0', '0', '0']
     ┌───┐      ░ ┌────────────┐ ░ ┌─┐      
q_0: ┤ H ├──────░─┤ PauliMeasZ ├─░─┤M├──────
     ├───┤      ░ ├────────────┤ ░ └╥┘┌─┐   
q_1: ┤ H ├──■───░─┤ PauliMeasZ ├─░──╫─┤M├───
     └───┘┌─┴─┐ ░ ├────────────┤ ░  ║ └╥┘┌─┐
q_2: ─────┤ X ├─░─┤ PauliMeasX ├─░──╫──╫─┤M├
          └───┘ ░ └────────────┘ ░  ║  ║ └╥┘
c_0: ═══════════════════════════════╩══

Map the outcomes of the QST experiments into sequences that can be used to train the Transformer model

In [14]:
# No. of measurement sets for Pauli-6 POVM
n_sets = 3**n_qubits

data = np.zeros((n_sets*n_shots, n_qubits), dtype=int)
idx = 0

for n_set in range(n_sets):
    # Raw measurement counts for 1 experiment e.g. {'0...0':X, '1...1':Y}
    meas_dict = counts_dict[n_set]
    
    # Measurement setup e.g. ['0', ..., '0'] -> [Z, ..., Z]
    meas_setting = header_dict[n_set]
    # Try the different method - meas_setting.reverse()
    
    for bitstring, count in meas_dict.items():
        # Map the projective outcomes to POVM outcomes
        # e.g. given setting [1...], map outcome [0..] to [2..] and [1..] to [3..]
        # outcome = [2*int(meas_setting[i]) + int(bitstring[i]) for i in range(n_qubits)]
        # Try the different mapping from the get_data notebook - totally works - reconstruction is good again
        outcome = [3*int(bitstring[(n_qubits-1)-i]) + int(meas_setting[i]) for i in range(n_qubits)]
        data[idx:idx+count] = np.array(outcome)
        idx += count

print('No. of measurements in the dataset', len(data))

No. of measurements in the dataset 2700


Save the generated dataset to disk

In [15]:
# save options
overwrite_data = True
if os.path.exists(f'{state_dir}/{n_qubits}_{len(data)}.npy'):
    if overwrite_data:
        print('overwriting existing dataset')
        np.save(f'{state_dir}/{n_qubits}_{len(data)}.npy', data)
else:
    print('creating new dataset')
    np.save(f'{state_dir}/{n_qubits}_{len(data)}.npy', data)

overwriting existing dataset


Run standard MLE

In [16]:
t1 = time.time()
try:
    import cvxpy

    # Set analysis option for cvxpy fitter
    qst_exp.analysis.set_options(fitter='cvxpy_linear_lstsq')
    
    # Re-run experiment
    qst_data_2 = qst_exp.run(sim_backend, seed_simulation=100).block_for_results()
except ModuleNotFoundError:
    print("CVXPY is not installed")
t = divmod(time.time() - t1, 60)
print(f'Linear MLE : took {t[0]} minutes {t[1]} seconds')



Linear MLE : took 1.0 minutes 7.71126651763916 seconds


In [17]:
# Save the constrained MLE dm to the disk
overwrite_mle = True
mle_dm_np = qst_data_2.analysis_results("state").value.data
mle_dm_path = f'{state_dir}/{n_qubits}_{(3**n_qubits)*n_shots}_linear_mle_dm.npy'

if os.path.exists(mle_dm_path):
    if overwrite_mle:
        print('overwriting existing mle dm')
        np.save(mle_dm_path, mle_dm_np)
else:
    print('saving new mle dm')
    np.save(mle_dm_path, mle_dm_np)

overwriting existing mle dm


Run Gaussian MLE

In [18]:
t1 = time.time()
try:
    import cvxpy

    # Set analysis option for cvxpy fitter
    qst_exp.analysis.set_options(fitter='cvxpy_gaussian_lstsq')
    
    # Re-run experiment
    qst_data_3 = qst_exp.run(sim_backend, seed_simulation=100).block_for_results()
except ModuleNotFoundError:
    print("CVXPY is not installed")
t = divmod(time.time() - t1, 60)
print(f'Gaussian MLE : took {t[0]} minutes {t[1]} seconds')

Gaussian MLE : took 0.0 minutes 6.17926287651062 seconds


In [19]:
# Save the constrained MLE dm to the disk
overwrite_mle = True
mle_dm_np = qst_data_3.analysis_results("state").value.data
mle_dm_path = f'{state_dir}/{n_qubits}_{(3**n_qubits)*n_shots}_gaussian_mle_dm.npy'

if os.path.exists(mle_dm_path):
    if overwrite_mle:
        print('overwriting existing mle dm')
        np.save(mle_dm_path, mle_dm_np)
else:
    print('saving new mle dm')
    np.save(mle_dm_path, mle_dm_np)

overwriting existing mle dm
